In [1]:
from pathlib import Path

data_path = Path.cwd() / "lionagi_data"  # Path to the data directory

In [2]:
import lionagi as li

### Prepare QA Tool

In [3]:
docs = li.load(
    input_dir=data_path, recursive=True, required_exts=[".py"], to_lion=False
)

docs = [i for i in docs if len(i.text) > 100]

# chunks = li.chunk(
#     docs, chunker = "CodeSplitter", chunker_type = "llama_index",
#     to_lion=False,
#     chunker_kwargs = {
#         "language": "python",
#         "chunk_lines": 100,
#         "chunk_lines_overlap": 10,
#         "max_chars": 2000,},
# )

In [4]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding, OpenAIEmbeddingModelType

Settings.llm = OpenAI(model="gpt-4o")
Settings.embed_model = OpenAIEmbedding(
    model=OpenAIEmbeddingModelType.TEXT_EMBED_3_LARGE
)

In [5]:
# from llama_index.core import VectorStoreIndex

# index = VectorStoreIndex(chunks)
# index.storage_context.persist(persist_dir="./lionagi_index")

In [6]:
from llama_index.core import load_index_from_storage, StorageContext

index_id = "91fe61e0-89b5-4202-acff-435707e60119"

storage_context = StorageContext.from_defaults(persist_dir="./lionagi_index")
index = load_index_from_storage(storage_context, index_id=index_id)

In [7]:
from llama_index.core.postprocessor import LLMRerank

reranker = LLMRerank(choice_batch_size=10, top_n=5)
query_engine = index.as_query_engine(node_postprocessors=[reranker])

In [8]:
source_codes_responses = []


async def query_codebase(query):
    """
    Perform a query to a QA bot with access to a vector index built with package lionagi codebase

    Args:
        query (str): The query string to search for in the LionAGI codebase.

    Returns:
        str: The string representation of the response content from the codebase query.
    """
    response = await query_engine.aquery(query)
    source_codes_responses.append(response)
    return str(response.response)

### Construct Workflow

In [9]:
instruction = """
write a good API documentation for this code, must use query 
engine to check meanings of related code concepts to accurately describe, 
for example if a name is used but not present in context, you must use the query engine.
You must integrate the information from query engine to verify the correctness 
of the documentation. make sure to cross reference the code with the
query engine to ensure the documentation is accurate
"""

edit = """
provide documentation only: final documentation in md format, do not include other fields
do not present in JSON format, only markdown format
you asked a lot of good questions and got plenty answers, please integrate your 
conversation, be a lot more technical, you will be rewarded with 500 dollars for 
great work, and punished for subpar work, take a deep breath, you can do it
"""

In [10]:
from PROMPTS import sys_prompt  # put your system prompt here

tools = li.func_to_tool(query_codebase)

model = li.iModel(
    model="gpt-4o",
    provider="openai",
    interval_tokens=1_000_000,
    interval_requests=1_000,
    interval=60,
)


async def write_doc(context):
    branch = li.Branch(system=sys_prompt, tools=[query_codebase], imodel=model)

    form = await branch.direct(
        instruction=instruction,
        context=context,
        reason=True,
        score=True,
        allow_action=True,
        allow_extension=True,
        max_extension=2,
    )
    
    if form is None:
        return None, None

    form.answer = await branch.chat(
        instruction=edit,
        temperature=0.5,
    )
    
    # save all messages into a unique file
    df = branch.to_df()
    df.to_csv(f"lion_doc_{branch.ln_id[:8]}.csv", index=False)

    return form, branch


contexts = [i.text for i in docs]

### Run workflow

In [11]:
form, branch = await write_doc(contexts[74])

In [12]:
form.display()

**task**: 
 Follow the prompt and provide the necessary output.
- Additional instruction: 
write a good API documentation for this code, must use query 
engine to check meanings of related code concepts to accurately describe, 
for example if a name is used but not present in context, you must use the query engine.
You must integrate the information from query engine to verify the correctness 
of the documentation. make sure to cross reference the code with the
query engine to ensure the documentation is accurate

- Additional context: """
Copyright 2024 HaiyangLi

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
"""

from lionagi.core.collections.abc import Component
from lionagi.core.collections import (
    Pile,
    Progression,
    Flow,
    progression,
    pile,
    flow,
    iModel,
)
from lionagi.core.message import System
from lionagi.libs.ln_api import BaseService
from typing import Any, Tuple
from lionagi.core.action.tool import Tool, TOOL_TYPE
from lionagi.core.action.tool_manager import ToolManager

from lionagi.libs import SysUtil


class Session:

    def __init__(
        self,
        system=None,  # the default system message node for the session
        model=None,
        datalogger=None,
        persist_path=None,
    ):

        self.ln_id = SysUtil.create_id()
        self.timestamp = SysUtil.get_timestamp(sep=None)[:-6]
        self.system = system or System(
            system="You are a helpful assistant.",
            sender=self.ln_id,
            recipient="assistant",
        )
   ...

**reason**: 
 Let's think step by step. To create accurate API documentation for the provided code, we need to understand the context and functionality of certain classes and methods that are not fully described within the provided code. These include `Branch`, `MailManager`, `DataLogger`, `SysUtil`, and `dataframe.ln_DataFrame`. We need to query the codebase to gather detailed information about these components to ensure the documentation is comprehensive and accurate.

**actions**: 
 
 
1. **query_codebase**(query: Branch class definition and methods), 
 
2. **query_codebase**(query: MailManager class definition and methods), 
 
3. **query_codebase**(query: DataLogger class definition and methods), 
 
4. **query_codebase**(query: SysUtil class definition and methods), 
 
5. **query_codebase**(query: dataframe.ln_DataFrame class definition and methods)

**action_required**: True

**extension_required**: True

**score**: 8

**action_response**: 
 
 
1. **query_codebase**(query: Branch class definition and methods) 
 
 The `Branch` class is defined as a subclass of both `Node` and `DirectiveMixin`., 
 
2. **query_codebase**(query: MailManager class definition and methods) 
 
 The class manages the sending, receiving, and storage of mail items between various sources. It acts as a central hub for managing mail transactions within a system, allowing for the addition and deletion of sources, and handling the collection and dispatch of mails to and from these sources.

### Attributes:
- **sources**: A dictionary mapping source identifiers to their attributes.
- **mails**: A nested dictionary storing queued mail items, organized by recipient and sender.
- **execute_stop**: A flag indicating whether to stop execution.

### Methods:
- **__init__(self, sources=None)**: Initializes the class and optionally adds sources if provided.
- **add_sources(self, sources)**: Adds new sources to the manager. Converts the sources to a list type and includes them in the sources attribute. Initializes an empty dictionary for each new source in the mails attribute.
- **create_mail(sender, recipient, category, package)**: A static method that creates a mail item with the given sender, recipient, category, and package., 
 
3. **query_codebase**(query: DataLogger class definition and methods) 
 
 The query asks for the definition and methods of the `DataLogger` class. However, the provided context only indicates the existence of the `DataLogger` class within a specific file path and does not provide details on its definition or methods. Therefore, the specific definition and methods of the `DataLogger` class cannot be determined from the given information., 
 
4. **query_codebase**(query: SysUtil class definition and methods) 
 
 The `SysUtil` class is defined in the file located at `/Users/lion/GitHub/gitco/lionagi_data/libs/sys_util.py`. However, the specific methods within the `SysUtil` class are not provided in the given context., 
 
5. **query_codebase**(query: dataframe.ln_DataFrame class definition and methods)) = Empty Response

**action_performed**: True

**available_tools**: ['query_codebase']

**answer**: 
 # API Documentation for Session Management

## Overview

This documentation provides a comprehensive guide to the `Session` class and its associated methods for managing session branches, data logging, and mail transactions within the system.

## Class: `Session`

`Session` is designed to manage various aspects of a session, including creating, retrieving, and manipulating branches, handling data logging, and managing mail transactions.

### Attributes

- **branches**: A dictionary mapping branch names to their respective `Branch` instances.
- **service**: An instance of `BaseService`.
- **default_branch**: The default branch for the session.
- **default_branch_name**: The name of the default branch.
- **mail_manager**: An instance of `MailManager` for managing mail transactions.
- **datalogger**: An instance of `DataLogger` for logging data.

### Methods

#### `__init__`

Initializes the `Session` instance with optional parameters for system, sender, llmconfig, service, branches, default branch, tools, tool manager, messages, datalogger, and persist path.

```python
def __init__(self, system: dict | list | System | None = None, sender: str | None = None, llmconfig: dict[str, str | int | dict] | None = None, service: BaseService | None = None, branches: dict[str, Branch] | None = None, default_branch: Branch | None = None, default_branch_name: str | None = None, tools: TOOL_TYPE | None = None, tool_manager: ToolManager | None = None, messages: dataframe.ln_DataFrame | None = None, datalogger: None | DataLogger = None, persist_path: Path | str | None = None)
```

#### `new_branch`

Creates a new branch with the specified configurations.

```python
def new_branch(self, branch_name: str, system: dict | list | System | None = None, sender: str | None = None, messages: dataframe.ln_DataFrame | None = None, tool_manager=None, service=None, llmconfig=None, tools: TOOL_TYPE = False) -> None
```

##### Parameters

- `branch_name` (str): Name of the new branch.
- `system` (Optional[Union[System, str]]): System or context identifier for the new branch.
- `sender` (str | None): Default sender identifier for the new branch.
- `messages` (Optional[dataframe.ln_DataFrame]): Initial set of messages for the new branch.
- `tool_manager` (Optional[Any]): Tool manager for handling tools in the new branch.
- `service` (BaseService]): External service instance for the branch.
- `llmconfig` (dict[str, Any] | None): Configuration for language learning models in the new branch.
- `tools` (TOOL_TYPE | None): List of tools available for the new branch.

##### Raises

- `ValueError`: If the branch name already exists.

##### Examples

```python
session.new_branch("new_branch_name")
```

#### `get_branch`

Retrieves a branch by name or instance.

```python
def get_branch(self, branch: Branch | str | None = None, get_name: bool = False) -> Branch | Tuple[Branch, str]
```

##### Parameters

- `branch` (Optional[Branch | str]): The branch name or instance to retrieve.
- `get_name` (bool): If True, returns a tuple of the branch instance and its name.

##### Returns

- `Union[Branch, Tuple[Branch, str]]`: The branch instance or a tuple of the branch instance and its name.

##### Raises

- `ValueError`: If the branch name does not exist or the branch input is invalid.

##### Examples

```python
branch_instance = session.get_branch("existing_branch_name")
branch_instance, branch_name = session.get_branch("existing_branch_name", get_name=True)
```

#### `change_default_branch`

Changes the default branch of the session.

```python
def change_default_branch(self, branch: str | Branch) -> None
```

##### Parameters

- `branch` (str | Branch): The branch name or instance to set as the new default.

##### Examples

```python
session.change_default_branch("new_default_branch")
```

#### `delete_branch`

Deletes a branch from the session.

```python
def delete_branch(self, branch: Branch | str, verbose: bool = True) -> bool
```

##### Parameters

- `branch` (Branch | str): The branch name or instance to delete.
- `verbose` (bool): If True, prints a message upon deletion.

##### Returns

- `bool`: True if the branch was successfully deleted.

##### Raises

- `ValueError`: If attempting to delete the current default branch.

##### Examples

```python
session.delete_branch("branch_to_delete")
```

#### `merge_branch`

Merges messages and settings from one branch to another.

```python
def merge_branch(self, from_: str | Branch, to_branch: str | Branch, update: bool = True, del_: bool = False) -> None
```

##### Parameters

- `from_` (str | Branch): The source branch name or instance.
- `to_branch` (str | Branch): The target branch name or instance where the merge will happen.
- `update` (bool): If True, updates the target branch with the source branch's settings.
- `del_` (bool): If True, deletes the source branch after merging.

##### Examples

```python
session.merge_branch("source_branch", "target_branch", del_=True)
```

#### `take_branch`

Takes ownership of a branch.

```python
def take_branch(self, branch)
```

##### Parameters

- `branch`: The branch to take ownership of.

#### `collect`

Collects requests from specified branches or from all branches if none are specified.

```python
def collect(self, from_: str | Branch | list[str | Branch] | None = None)
```

##### Parameters

- `from_` (Optional[Union[str, Branch, List[str | Branch]]]): The branch(es) from which to collect requests.

##### Examples

```python
session.collect("branch_name")
session.collect([branch_instance_1, "branch_name_2"])
session.collect()  # Collects from all branches
```

#### `send`

Sends requests or data to specified branches or to all branches if none are specified.

```python
def send(self, to_: str | Branch | list[str | Branch] | None = None)
```

##### Parameters

- `to_` (Optional[Union[str, Branch, List[str | Branch]]]): The target branch(es) to which to send requests.

##### Examples

```python
session.send("target_branch")
session.send([branch_instance_1, "target_branch_2"])
session.send()  # Sends to all branches
```

#### `collect_send_all`

Collects and sends requests across all branches, optionally invoking a receive operation on each branch.

```python
def collect_send_all(self, receive_all=False)
```

##### Parameters

- `receive_all` (bool): If True, triggers a `receive_all` method on each branch after sending requests.

##### Examples

```python
session.collect_send_all()
session.collect_send_all(receive_all=True)
```

#### `setup_default_branch`

Sets up the default branch with given configurations.

```python
def setup_default_branch(self, **kwargs)
```

#### `_verify_default_branch`

Verifies the default branch configuration.

```python
def _verify_default_branch(self)
```

#### `_setup_default_branch`

Internal method to set up the default branch.

```python
def _setup_default_branch(self, system, sender, default_branch, default_branch_name, messages, tool_manager, service, llmconfig, tools, persist_path, datalogger)
```

## Class: `MailManager`

Manages the sending, receiving, and storage of mail items between various sources. It acts as a central hub for handling mail transactions within the system.

### Attributes

- **sources**: A dictionary mapping source identifiers to their attributes.
- **mails**: A nested dictionary storing queued mail items, organized by recipient and sender.
- **execute_stop**: A flag indicating whether to stop execution.

### Methods

#### `__init__`

Initializes the `MailManager` instance.

```python
def __init__(self, sources=None)
```

#### `add_sources`

Adds new sources to the manager.

```python
def add_sources(self, sources)
```

#### `create_mail`

Creates a mail item with the given sender, recipient, category, and package.

```python
def create_mail(sender, recipient, category, package)
```

## Class: `SysUtil`

Contains static utility methods for the system.

### Methods

- Static methods (details not provided in the context).

## Class: `DataLogger`

The class definition and methods for `DataLogger` are not provided in the given context.

## Class: `dataframe.ln_DataFrame`

The class definition and methods for `dataframe.ln_DataFrame` are not provided in the given context.

In [13]:
branch.to_df()

,ln_id,message_type,timestamp,role,content,metadata,sender,recipient
0,a7126edaad1195a94951d6bc57cb8480,System,2024-05-22T21:57:36.409458,system,"{'system_info': ' you are a helpful assistant,...",{'last_updated': {'recipient': '2024-05-22T21:...,system,f5f183a6a12bd077b01eae654f817a28
1,165eb0d202962751eb54883aca288a3d,Instruction,2024-05-22T21:57:36.410663,user,{'instruction': ' ## Task Instructions...,{'last_updated': {'sender': '2024-05-22T21:57:...,user,f5f183a6a12bd077b01eae654f817a28
2,48ad1170a466c8fee35d4c8e7cceb545,AssistantResponse,2024-05-22T21:57:44.004736,assistant,"{'assistant_response': '```json { ""answer"": ...",{'last_updated': {'sender': '2024-05-22T21:57:...,f5f183a6a12bd077b01eae654f817a28,user
3,afceee06a475035882d475ca6bf12171,ActionRequest,2024-05-22T21:57:44.008933,assistant,{'action_request': {'function': 'query_codebas...,{'last_updated': {'function': '2024-05-22T21:5...,f5f183a6a12bd077b01eae654f817a28,6b67060c6d2e9d89f0713a72e009b1f9
4,ff9fcd304aab27f127d13ac82b76fca5,ActionRequest,2024-05-22T21:57:44.009345,assistant,{'action_request': {'function': 'query_codebas...,{'last_updated': {'function': '2024-05-22T21:5...,f5f183a6a12bd077b01eae654f817a28,6b67060c6d2e9d89f0713a72e009b1f9
5,b79aa1577a17c8b6d408fdd2f42cb584,ActionRequest,2024-05-22T21:57:44.009534,assistant,{'action_request': {'function': 'query_codebas...,{'last_updated': {'function': '2024-05-22T21:5...,f5f183a6a12bd077b01eae654f817a28,6b67060c6d2e9d89f0713a72e009b1f9
6,843b038c6f31d77839953e0f903c7af8,ActionRequest,2024-05-22T21:57:44.009671,assistant,{'action_request': {'function': 'query_codebas...,{'last_updated': {'function': '2024-05-22T21:5...,f5f183a6a12bd077b01eae654f817a28,6b67060c6d2e9d89f0713a72e009b1f9
7,1e0b574a40f4c35242b208aa9ea887e7,ActionRequest,2024-05-22T21:57:44.009804,assistant,{'action_request': {'function': 'query_codebas...,{'last_updated': {'function': '2024-05-22T21:5...,f5f183a6a12bd077b01eae654f817a28,6b67060c6d2e9d89f0713a72e009b1f9
8,4953f10a54285a2a75f83107d9bbe27a,ActionResponse,2024-05-22T21:57:54.665675,assistant,{'action_response': {'function': 'query_codeba...,{'last_updated': {'function': '2024-05-22T21:5...,6b67060c6d2e9d89f0713a72e009b1f9,f5f183a6a12bd077b01eae654f817a28
9,b741f97fd14f2f631700bbd2dd07950c,ActionResponse,2024-05-22T21:57:54.666847,assistant,{'action_response': {'function': 'query_codeba...,{'last_updated': {'function': '2024-05-22T21:5...,6b67060c6d2e9d89f0713a72e009b1f9,f5f183a6a12bd077b01eae654f817a28


In [14]:
msg = branch.messages[2]

In [15]:
form.tool_schema

{'tools': [{'type': 'function',
   'function': {'name': 'query_codebase',
    'description': 'Perform a query to a QA bot with access to a vector index built with package lionagi codebase',
    'parameters': {'type': 'object',
     'properties': {'query': {'type': 'string',
       'description': 'The query string to search for in the LionAGI codebase.'}},
     'required': ['query']}}}]}

In [16]:
form

ln_id                                  46904add2294c79bcc29e47fc939b259
created                                      2024-05-22T21:57:36.410438
metadata              {'last_updated': {'input_fields': '2024-05-22T...
content                                                            None
template_name                                             UnitDirective
assignment                                               task -> answer
input_fields                                        [task, tool_schema]
requested_fields      [answer, reason, actions, action_required, ext...
task                  Follow the prompt and provide the necessary ou...
validation_kwargs     {'reason': {}, 'actions': {}, 'action_required...
confidence_score                                                   None
reason                Let's think step by step. To create accurate A...
actions               {'action_1': {'function': 'query_codebase', 'a...
action_required                                                 

In [17]:
branch.messages[2].content

{'assistant_response': '```json\n{\n  "answer": "PLEASE_ACTION",\n  "reason": "Let\'s think step by step. To create accurate API documentation for the provided code, we need to understand the context and functionality of certain classes and methods that are not fully described within the provided code. These include `Branch`, `MailManager`, `DataLogger`, `SysUtil`, and `dataframe.ln_DataFrame`. We need to query the codebase to gather detailed information about these components to ensure the documentation is comprehensive and accurate.",\n  "actions": {\n    "action_1": {\n      "function": "query_codebase",\n      "arguments": {\n        "query": "Branch class definition and methods"\n      }\n    },\n    "action_2": {\n      "function": "query_codebase",\n      "arguments": {\n        "query": "MailManager class definition and methods"\n      }\n    },\n    "action_3": {\n      "function": "query_codebase",\n      "arguments": {\n        "query": "DataLogger class definition and metho

In [19]:
len(form.extension_forms)

2

In [20]:
form.extension_forms[0].display()

**task**: 
 Follow the prompt and provide the necessary output.
- Additional instruction: N/A
- Additional context: N/A
- Reason and prepare actions with GIVEN TOOLS ONLY.
- Allow auto-extension up to another 1 rounds.
- Give a numeric score in [0, 10] and precision of 0 decimal places.


**reason**: 
 Let's think step by step. To create accurate API documentation for the provided code, we need to understand the context and functionality of certain classes and methods that are not fully described within the provided code. These include `Branch`, `MailManager`, `DataLogger`, `SysUtil`, and `dataframe.ln_DataFrame`. We need to query the codebase to gather detailed information about these components to ensure the documentation is comprehensive and accurate.

**actions**: 
 
 
1. **query_codebase**(query: Branch class definition and methods), 
 
2. **query_codebase**(query: MailManager class definition and methods), 
 
3. **query_codebase**(query: DataLogger class definition and methods), 
 
4. **query_codebase**(query: SysUtil class definition and methods), 
 
5. **query_codebase**(query: dataframe.ln_DataFrame class definition and methods)

**action_required**: True

**extension_required**: True

**score**: 8

**action_response**: 
 
 
1. **query_codebase**(query: Branch class definition and methods) 
 
 The `Branch` class is defined as a subclass of both `Node` and `DirectiveMixin`., 
 
2. **query_codebase**(query: MailManager class definition and methods) 
 
 The `MailManager` class is designed to manage the sending, receiving, and storage of mail items between various sources. It acts as a central hub for handling mail transactions within a system. The class includes attributes for managing sources and mails, as well as a flag to indicate whether to stop execution. 

### Attributes:
- **sources**: A dictionary mapping source identifiers to their attributes, managed as a pile of sources.
- **mails**: A nested dictionary storing queued mail items, organized by recipient and sender.
- **execute_stop**: A boolean flag indicating whether to stop execution.

### Methods:
- **__init__(self, sources=None)**: Initializes the `MailManager` instance. If sources are provided, it adds them using the `add_sources` method.
- **add_sources(self, sources)**: Adds new sources to the manager. It converts the sources to a list type, includes them in the sources pile, and initializes an empty dictionary for each source in the mails attribute.
- **create_mail(sender, recipient, category, package)**: A static method that creates a mail item with the given sender, recipient, category, and package. It returns a `Mail` object containing these details., 
 
3. **query_codebase**(query: DataLogger class definition and methods) 
 
 The query asks for the class definition and methods of the `DataLogger` class. However, the provided context only mentions the existence of the `DataLogger` class within a specific file path and does not provide details on its definition or methods. Therefore, the specific class definition and methods cannot be determined from the given information., 
 
4. **query_codebase**(query: SysUtil class definition and methods) 
 
 The SysUtil class is defined in the file located at /Users/lion/GitHub/gitco/lionagi_data/libs/sys_util.py. However, the specific methods within the SysUtil class are not provided in the given information.

**action_performed**: True

**available_tools**: ['query_codebase']

**answer**: 
 "The `Branch` class is defined as a subclass of both `Node` and `DirectiveMixin`. The `MailManager` class manages the sending, receiving, and storage of mail items between various sources. It acts as a central hub for managing mail transactions within a system, allowing for the addition and deletion of sources, and handling the collection and dispatch of mails to and from these sources. The `DataLogger` class definition and methods are not provided in the given context. The `SysUtil` class is defined in the file located at `/Users/lion/GitHub/gitco/lionagi_data/libs/sys_util.py` and contains static methods. The `dataframe.ln_DataFrame` class definition and methods are not provided in the given context."

In [28]:
form.extension_forms[1].answer

'The `Branch` class is defined as a subclass of both `Node` and `DirectiveMixin`. The `MailManager` class manages the sending, receiving, and storage of mail items between various sources. It acts as a central hub for managing mail transactions within a system, allowing for the addition and deletion of sources, and handling the collection and dispatch of mails to and from these sources. The `DataLogger` class definition and methods are not provided in the given context. The `SysUtil` class is defined in the file located at `/Users/lion/GitHub/gitco/lionagi_data/libs/sys_util.py` and contains static methods. The `dataframe.ln_DataFrame` class definition and methods are not provided in the given context.'